In [46]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [47]:
BASE_URL = "https://www.ttmg.org/insidersguide/new-york-mta-bus-roster/new-york-mta-bus-roster-depot/"
html = requests.get(DEPOT_URL).text
soup = BeautifulSoup(html, "html5lib")

In [55]:
divisions = list()
for division in soup.findAll("span", {"class": "mw-headline"}):
    divisions.append(division.getText().strip())
print(divisions)

['Bronx Division', 'Manhattan Division', 'Queens Division', 'Brooklyn Division', 'Staten Island Division']


In [127]:
bronx_df = manhattan_df = queens_df = brooklyn_df = si_df = pd.DataFrame()
bronx, manhattan, queens, brooklyn, si = soup.findAll("table", {"style": "border-spacing: 2px; border: 3px solid white; font-size: 90%; width: 100%"})
boroughs = {"Bronx": bronx, "Manhattan": manhattan, "Queens": queens, "Brooklyn": brooklyn, "Staten Island": si}

In [132]:
import re

def save_depot_information(boroughs):
    df = pd.DataFrame(columns=["Borough", "Address", "Total buses"])
    for borough_name in boroughs:
        borough = boroughs[borough_name]
        for facility in borough.findAll("tr", {"style": "background:#000000; color:#FFFFFF", "align": "left"}):
            data = facility.find("td").getText()
            information = data.split("\n")
            name, address, total_buses = information[1], re.sub(r"\([^)]*\), ", "", information[2]), int(information[3][information[3].rfind(" ") + 1:])
            df.loc[name, ["Borough", "Address", "Total buses"]] = borough_name, address, total_buses
    df.index.name = "Depot"
    return df

In [134]:
depots_df = save_depot_information(boroughs)
depots_df.head()

,Borough,Address,Total buses
Depot,,,
Eastchester Depot (EC),Bronx,"3320 Tillotson Av, Bronx, NY 10475",142
Gun Hill Depot (GH),Bronx,"1910 Bartow Av, Bronx, NY 10469",284
Kingsbridge Depot (KB),Bronx,"4065 10th Av, New York, NY 10034",272
West Farms Depot (WF),Bronx,"Bronx Division Headquarters, 1100 E 177th St, ...",321
Yonkers Depot (YO),Bronx,"59 Babcock Pl, Yonkers, NY 10701",84


In [136]:
depots_df.to_csv("depot_data/depots.csv")